In [3]:
import json
import requests
from xml.etree import ElementTree

url = "http://localhost:9003/?output_format=jsonl"

text = "A mogę, bo moim zdaniem jest do niczego. I to delikatnie mówiąc... Rzecz gustu :) Ja sobie z przyjemnością obejrzałam wczoraj \"Live and Let Die\". Nareszcie bez Connery'ego!!! Pasuje. Najbardziej ze wszystkich. Może trochę za ładny, ale grą pasuje. IMHO. A Connery to nie święta krowa ;)"

In [38]:
def parse_line(line):
    r = requests.post(url, data = line.encode('utf-8'))
    out = json.loads(r.text)
    return out

flatten = lambda t: [item for sublist in t for item in sublist]

In [24]:
with open('train-raw.txt') as f:
    lines = list(filter(lambda x: x != "", [line.rstrip() for line in f]))
tree = ElementTree.parse('train-gold.xml')
root = tree.getroot()

In [51]:
chunks = tree.find("chunkList").findall("chunk")

matched = 0
miss_matched = 0
CTAG_POSSITION = 2
BASE_POSSITION = 1
TEXT_POSSITION = 0

chunk_id = 0
for line in chunks:
    curr_line = flatten(parse_line(lines[chunk_id]))
#     print(len(curr_line), curr_line)
    word_id = 0
    
    for sentence in line.findall("chunk"):
        
        for word in sentence.findall("tok"):
            if word_id >= len(curr_line):
                miss_matched += 1
                break
            
            curr_word = curr_line[word_id]
            orth = word.find("orth").text
            base = word.find("lex").find("base").text
            ctag = word.find("lex").find("ctag").text

#             print(f"{orth} {base} {ctag}")
#             print(curr_word)
            if ctag == curr_word[CTAG_POSSITION]:
                matched += 1
            else:
#                 print(orth, ctag)
#                 print(curr_word[TEXT_POSSITION], curr_word[CTAG_POSSITION])
                miss_matched += 1
            word_id += 1
            
    chunk_id += 1

In [52]:
print(matched, miss_matched)

1121365 93709


In [53]:
matched/(matched+miss_matched)

0.9228779481743499